In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

import numpy as np
from scipy.spatial.distance import cdist

In [10]:
clients = pd.read_csv('./hack_data/clients.csv')
materials = pd.read_csv('./hack_data/materials.csv')
plants = pd.read_csv('./hack_data/plants.csv')
transaction = pd.read_parquet('./hack_data/transactions.parquet')#[:10000]


In [48]:
class Recomender:
    def __init__(self, transaction, emb_size=64):
        self.emb_size = emb_size
        self.client_embs, self.mat_embs, self.mat2id, self.client2id = self.get_embeddings(transaction)
        self.id2mat = dict((j, pl) for (pl, j) in self.mat2id.items())
        self.id2client = dict((j, cl) for (cl, j) in self.client2id.items())
        
    def get_embeddings(self, transaction):
        mat2id = dict((p,j) for j, p in enumerate(set(transaction.material)))
        client2id = dict((c,j) for j, c in enumerate(set(transaction.client_id)))
        gr = transaction.groupby(['material', 'client_id']).apply(lambda x: len(x))

        M = np.zeros((len(client2id), len(mat2id)))

        for (pl, cl), n in gr.iteritems():
            M[client2id[cl], mat2id[pl]] = n

        u, s, v = np.linalg.svd(M, full_matrices=False)
        u = u * s[None]
        u = u[:, :self.emb_size]
        v = v[:self.emb_size]

        u = u / np.linalg.norm(u, axis=-1)[:,None]
        v = v / np.linalg.norm(v, axis=-1)[:,None]

        return u, v.T, mat2id, client2id
    
    def get_material_chq(self, chq_materials):
        chq_emb = np.array([self.mat_embs[recomender.mat2id[mat]] for mat in chq_materials])

        d = cdist(chq_emb, self.mat_embs, metric='cosine').mean(0)
        order = np.argsort(d)
        closest_materials = [self.id2mat[j] for j in order]

        for mat in closest_materials:
            if mat not in chq_materials:
                return mat

In [49]:
recomender = Recomender(transaction)


<ipython-input-48-cfc3a171ded9>:23: RuntimeWarning: invalid value encountered in true_divide
  u = u / np.linalg.norm(u, axis=-1)[:,None]


In [53]:
chqs = list(set(transaction.chq_id))
chq_materials = transaction[transaction.chq_id == chqs[11]].material

recomender.get_material_chq(list(chq_materials))

'aa793dd5e70b40dcae2de57b001beb7a'